## Pré-processamento

In [154]:
import pandas as pd
from collections import Counter
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
import string
from sklearn.model_selection import train_test_split
#import spacy

df = pd.read_csv('factnews_dataset.csv')

#criar grupo treino e teste
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

stop_words = set(stopwords.words('portuguese'))

#criar coluna tokens
train_df['tokens'] = train_df['sentences'].apply(lambda x: nltk.word_tokenize(str(x).lower()))

# Tokenizar sem stopwords
train_df['tokens_no_stopwords'] = train_df['tokens'].apply(lambda toks: [t for t in toks if t not in stop_words])

## Detetar palavras únicas entre as classes

In [155]:
classes = df['classe'].unique()

def palavras_unicas_por_classe(df, token_col):
    
    # contar tokens por classe
    tokens_por_classe = {}
    for classe in classes:
        linhas_classe = df[df['classe'] == classe][token_col].dropna()
        contagem = {}
        for lista in linhas_classe:
            for token in lista:
                if token in contagem:
                    contagem[token] += 1
                else:
                    contagem[token] = 1
        tokens_por_classe[classe] = contagem

    # detetar palavras exclusivas por classe
    palavras_unicas = {}
    for classe in classes:
        # lista de palavras das outras classes
        outras_palavras = []
        for outra_classe in classes:
            if outra_classe != classe:
                for token in tokens_por_classe[outra_classe]:
                    outras_palavras.append(token)

        # dicionário palavras exclusivas com contagens
        unicas_classe = {}
        for palavra, contagem in tokens_por_classe[classe].items():
            exclusiva = True
            for p in outras_palavras:
                if p == palavra:
                    exclusiva = False
                    break
            if exclusiva:
                unicas_classe[palavra] = contagem

        palavras_unicas[classe] = unicas_classe

    return palavras_unicas

palavras_unicas_por_classe(train_df,'tokens')


{-1: {'repensado': 1,
  'crescem': 1,
  'catástrofe': 1,
  'precedentes': 1,
  'vocês': 15,
  'preocupados': 3,
  'questionar': 1,
  'valeria': 2,
  'combati': 2,
  'gente': 33,
  'tranquilidade': 1,
  'eu': 122,
  'saberia': 1,
  'fique': 2,
  'impunidade': 1,
  'admirável': 1,
  'escreva': 1,
  'tínhamos': 2,
  'vivemos': 1,
  'boa-fé': 1,
  'consideramos': 3,
  'proporções': 3,
  'monumentais': 3,
  'foi-se': 1,
  'acabara': 1,
  'ajudará': 2,
  'colaterais': 2,
  '!': 18,
  'pergunto': 4,
  'queremos': 6,
  'empinando': 1,
  'caramba': 1,
  'vamos': 20,
  'bater': 3,
  'gritou': 1,
  'tentamos': 3,
  'repetiram': 1,
  'realizamos': 1,
  'lealdade': 1,
  'pesada': 1,
  'sequenciamento': 1,
  'globalmente': 1,
  'lindmeyer': 1,
  'extinto': 1,
  'mentirosa': 1,
  'desmonte': 1,
  'evangélico': 1,
  'valle': 1,
  'sentimos': 2,
  'stewart': 1,
  'dissemos': 2,
  'paliativa': 1,
  'somos': 4,
  'heterogêneos': 1,
  'umas': 9,
  'quiser': 5,
  'tocado': 1,
  'divergem': 1,
  'declarada'

## Contruir regras para classificar o texto

In [156]:

def citacao(text):
    return '"' in str(text) or '“' in str(text) or '”' in str(text) or '[' in str(text) 

def bias(text):
    return '?' in str(text)

def classificar(row):
    if citacao(row['sentences']):
        return -1  
    if bias(row['sentences']):
        return 1
    return 0

ver verbo verbo numa frase - prob um bias

## Aplicar classificação no conjunto de teste

In [ ]:
test_df['Previsto'] = test_df.apply(classificar, axis=1)


## Avaliar as regras de classificação

In [158]:
print(test_df[['sentences', 'classe', 'Previsto']].head())

                                              sentences  classe  Previsto
996   No primeiro turno, o candidato da esquerda chi...       0         0
468   Esta é a segunda vez em menos de um ano que o ...       0         0
6077  A margem de erro é dois pontos para mais ou pa...       0         0
2127  Passei um dia na cidade nesta semana -moro no ...      -1         0
2080  Na ocasião, o peemedebista discordou dos outro...       0         0


In [ ]:
certos = (test_df['classe'] == test_df['Previsto']).sum()
total = len(df)
print(f'Certos: {certos} de {total}')
print(f'Accuracy: {certos/total:.2%}')

for c in classes:
    linhas_classe = test_df[test_df['classe'] == c]     
    total_classe = len(linhas_classe)
    certos_classe = (linhas_classe['classe'] == linhas_classe['Previsto']).sum()
    taxa_acerto = certos_classe / total_classe 
    print(f"Classe {c}: ", taxa_acerto*100, "%")

Certos: 969 de 6191
Accuracy: 15.65%
Classe -1:  65.21739130434783 %
Classe 0:  92.9245283018868 %
Classe 1:  0.8695652173913043 %
